In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
#from localcider.sequenceParameters import SequenceParameters
import scipy
from scipy.spatial.distance import cdist
import math

In [19]:
def split_sphere(center ,R = 1, horizontal_split = 36, vertical_split = 36):
    theta = np.linspace(0,360,horizontal_split+1)
    c = np.linspace(-1, 1, vertical_split+1)
    s = 1 - c**2
    x = R * np.outer(s, np.cos(theta))
    x += center[0]
    y = R * np.outer(s, np.sin(theta))
    y += center[1]
    z = R * np.outer(c, np.ones(horizontal_split+1))
    z += center[2]
    return x, y, z

In [83]:
pdb_path = './data/pdbs'
with open(os.path.join(pdb_path, 'AF-C0H3V8-F1.pdb')) as f:
    Atom_array = np.empty((0, 4))
    VdW_radii = {'C': 3.1, 'N': 2.95, 'O': 2.96} # Van der Waals radii in Angstrom enlarged by watermolecule radius 1.4 A (https://academic.oup.com/nar/article/49/W1/W559/6279848#267025710)
    C_C = 6.2
    C_N = 6.05
    C_O = 6.06
    N_N = 5.9
    N_O = 5.91
    O_O = 5.92
    X_array = np.array([[]])
    C_array = np.empty((0,4))
    N_array = np.empty((0,4))
    O_array = np.empty((0,4))
    for line in f:
        line = line.strip()
        if line.startswith('ATOM'):
            if ('ASP' in line and 'OD' in line) or ('GLU' in line and 'OE' in line):
                line_array = np.array([[line[7:12].strip(), line[27:38].strip(), line[39:46].strip(), line[47:54].strip()]])
                line_array = line_array.astype('float64')
                Atom_array = np.append(Atom_array, line_array, axis = 0)
                if 'C' in line[12:17]:
                    X_array = np.append(X_array, int('0'))
                elif 'N' in line[12:17]:
                    X_array = np.append(X_array, int('1'))
                elif 'O' in line[12:17]:
                    X_array = np.append(X_array, int('2'))
from function import distance
Atom_distance = distance(Atom_array, Atom_array, 6, remove_nan = False)
Atom_distance = np.where(Atom_distance==0, np.nan, Atom_distance)

In [84]:
def Sphere_coord(Atom_distance, Atom_coord):
    Sphere = {}
    for n in  np.unique(np.where(~np.isnan(Atom_distance[1:,1:]))[0]):
        xyz = Atom_array[n,1:4]
        x,y,z = split_sphere(xyz, R = 6)
        Sphere[int(Atom_array[n,0])] = np.array([x,y,z])
    return Sphere   
 
    

In [128]:
def intersect_vol(array:np.array, r1:str, r2:str):
    r1 = float(r1)
    r2 = float(r2)
    vol = array
    d = array[1:,1:]
    vol[1:,1:] = (np.pi*(r2+r1-d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2))/(12*d)
    return vol
#intersect_vol('1', '6', '6')

In [129]:
test = intersect_vol(Atom_distance, '6', '6')

print(test)


[[         nan  20.          21.          37.          38.
   76.          77.         100.         101.         257.
  258.         308.         309.         401.         402.
  421.         422.         517.         518.         526.
  527.        ]
 [ 20.                  nan 656.46220938          nan 315.33853324
           nan          nan          nan          nan          nan
           nan          nan          nan          nan          nan
           nan          nan          nan          nan          nan
           nan]
 [ 21.         656.46220938          nan          nan          nan
           nan          nan          nan          nan          nan
           nan          nan          nan          nan          nan
           nan          nan          nan          nan          nan
           nan]
 [ 37.                  nan          nan          nan 653.11078945
           nan 319.06345446          nan          nan          nan
           nan          nan          nan      

In [ ]:
def VdW_interaction(path, pdb_files=None):
    import numpy as np
    import os
    import scipy
    from scipy.spatial.distance import cdist
    if pdb_files is None:
        pdb_files = [f for f in os.listdir(path) if f.endswith('.pdb')]
    if isinstance(pdb_files, str):
        pdb_files = [pdb_files]
    Salt_bridges = dict()
    for pdb_file in pdb_files:
        Atom_array = np.empty((0,4))

    with open(os.path.join(pdb_path, 'AF-C0H3V8-F1.pdb')) as f:
        Atom_array = np.empty((0, 4))
        #VdW_radii = {'C': 3.1, 'N': 2.95, 'O': 2.96} # Van der Waals radii in Angstrom enlarged by watermolecule radius 1.4 A (https://academic.oup.com/nar/article/49/W1/W559/6279848#267025710)
        #C_C = 6.2
        #C_N = 6.05
        #C_O = 6.06
        #N_N = 5.9
        #N_O = 5.91
        #O_O = 5.92
        X_array = np.array([[]])
        for line in f:
            line = line.strip()
            if line.startswith('ATOM'):
                if ('ASP' in line and 'OD' in line) or ('GLU' in line and 'OE' in line):
                    line_array = np.array([[line[7:12].strip(), line[27:38].strip(), line[39:46].strip(), line[47:54].strip()]])
                    line_array = line_array.astype('float64')
                    Atom_array = np.append(Atom_array, line_array, axis = 0)
                    if 'C' in line[12:17]:
                        X_array = np.append(X_array, int('0'))
                    elif 'N' in line[12:17]:
                        X_array = np.append(X_array, int('1'))
                    elif 'O' in line[12:17]:
                        X_array = np.append(X_array, int('2'))
        
        from function import distance
        Atom_distance = distance(Atom_array, Atom_array, 6, remove_nan = False)
        Atom_distance = np.where(Atom_distance==0, np.nan, Atom_distance)